In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install datasets transformers evaluate accelerate

In [ ]:
# https://www.kaggle.com/competitions/nlp-getting-started

import numpy as np 
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import pandas as pd

train_dataset = load_dataset('csv', data_files='train.csv')
test_dataset = load_dataset('csv', data_files='test.csv')

train_dataset = train_dataset.rename_column("target", "labels")

# Shuffle the training dataset so no order effects
train_dataset = train_dataset.shuffle(seed=1) 
train_dataset = train_dataset.flatten_indices()

# Split into validation and training
split_dataset = train_dataset['train'].train_test_split(test_size=0.1)
train_dataset, valid_dataset = split_dataset["train"], split_dataset["test"]

# Tokenize training, test, and validation
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def tokenize(row):
    return tokenizer(row["text"], truncation=True, padding="max_length", max_length=256)
train_dataset = train_dataset.map(tokenize, batched=True)
valid_dataset = valid_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metrics.compute(predictions=predictions, references=labels)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()
trainer.push_to_hub()

preds = trainer.predict(test_dataset['train'])
preds = np.argmax(preds.predictions, axis=1)

submission_df = pd.DataFrame({
    "id": test_dataset["id"], 
    "target": preds
})
submission_df.head(10)
submission_df.to_csv("submit.csv", index=None)

# Improvements?
#TODO: consider if stratification during training would help with F1.
#TODO: look at other ppl's notebooks that used HF Transformers and see how I can improve score.
#TODO: see if including the fields beyond the text of the tweet impacts F1 score or not.  Let's do first pass without that stuff and see how leaderboard goes.